### Feature Selection 

Dataset: Ad-data_prep.csv

Import libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [ ]:
# Set Options for display
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100
pd.options.display.float_format = '{:.2f}'.format

Load the dataset

In [ ]:
df = pd.read_csv("../datasets/Ad-data_prep.csv", index_col = 0)

In [ ]:
df.head()

In [ ]:
df.shape

Separate Features from Target

In [ ]:
features = df.drop(["clicks","clicks_log"], axis=1)
target = df[["clicks","clicks_log"]]

Feature Selection: Low Variance Filter

In [ ]:
# Compute the variance and sort
features.var().sort_values()[:60]

In [ ]:
# Import the VarianceThreshold Function
from sklearn.feature_selection import VarianceThreshold

In [ ]:
# Instantiate the Function and Set the Threshold
selector = VarianceThreshold(0.01)

In [ ]:
# Apply the Function to filter out the Low Variance Columns/Features

filtered_data = selector.fit_transform(features)


filtered_features = pd.DataFrame(filtered_data, index=features.index)

In [ ]:
features.shape

In [ ]:
filtered_data.shape

In [ ]:
# Note, the DataFrame Created has no Column Names
filtered_features.head()

In [ ]:
# Use this attribute to get Column Names
selected = selector.get_support()

# Rename the columns
filtered_features.columns = features.columns[selected]

In [ ]:
features.columns

In [ ]:
filtered_features.head()

In [ ]:
filtered_features.var().sort_values().head()

In [ ]:
# Compare previous vs current number of Features
print("No. of Features (Original): %i" %len(features.columns))
print("No. of Features (Variance Filter): %i" %len(filtered_features.columns))

Feature Selection: High Correlation (Multicollinearity) Filter

Correlation Filter Function

In [ ]:
# Create correlation matrix
corr_matrix = filtered_features.corr().abs()

In [ ]:
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype('bool'))

In [ ]:
# Set the threshold
threshold = 0.90

In [ ]:
# Find index of feature columns with correlation greater than threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

In [ ]:
# View Features to Drop
to_drop

In [ ]:
len(to_drop)

In [ ]:
# Drop Features
filtered_features_2 = filtered_features.drop(to_drop, axis=1)

In [ ]:
# Compare previous vs current number of Features
print("No. of Features (Original): %i" %len(features.columns))
print("No. of Features (Variance Filter): %i" %len(filtered_features.columns))
print("No. of Features (Correlation Filter): %i" %len(filtered_features_2.columns))

Custom Function

In [ ]:
def correlation_filter(df,threshold):
    # Create correlation matrix
    corr_matrix = df.corr().abs()
    
    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype('bool'))
    
    # Find index of feature columns with correlation greater than threshold
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    
    return to_drop

In [ ]:
# Test the function
function_test = correlation_filter(filtered_features,threshold)

In [ ]:
len(function_test)

In [ ]:
function_test

Verification

In [ ]:
# Print out top correlated features

# Reshape the Matrix
correlated = corr_matrix.unstack()

# Reset Index from Multi-index to single index
correlated = correlated.reset_index(level=0).reset_index()

# Rename Columns
correlated.columns = ["Feature1", "Feature2", "Correlation"]

# Sort by Correlation Value
corr_sorted = correlated.sort_values("Correlation", ascending=False)

In [ ]:
correlated.head()

In [ ]:
# Remove Self-Correlated Rows
corr_sorted_pairs = corr_sorted[corr_sorted['Feature1'].values != corr_sorted['Feature2'].values]

In [ ]:
# Reset Index to easily filter out duplicate rows
corr_sorted_pairs.reset_index(drop=True,inplace=True)

In [ ]:
# Skip Odd Rows to Retain only 1 pair combination
corr_sorted_final = corr_sorted_pairs.iloc[::2]

In [ ]:
corr_sorted_final.Feature1.nunique()

In [ ]:
# Select rows based on threshold
corr_sorted_final_ver = corr_sorted_final[corr_sorted_final.Correlation > threshold]

In [ ]:
corr_sorted_final_ver

In [ ]:
to_drop

View the final file and save to csv

In [ ]:
# Compare previous vs current number of Features
print("No. of Features (Original): %i" %len(features.columns))
print("No. of Features (Variance Filter): %i" %len(filtered_features.columns))
print("No. of Features (Correlation Filter): %i" %len(filtered_features_2.columns))

In [ ]:
df_final = filtered_features_2.join(target)

In [ ]:
df_final.shape

In [ ]:
df_final.head()